In [100]:
# Importação das bibliotecas necessárias
import spacy
import re
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import TruncatedSVD
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
import numpy as np


In [2]:
path = "data/sentenças_educacao.xlsx"

# Part 1 - LLM

## Input de Docs com prompt
Input dos docs em um modelo LLM com prompt dedicado a clusterizar os textos da melhor forma possível explicando cada tópico e dando output do número de tópicos.

In [4]:
# Will be applied after due to cost of running an LLM locally

# Will be assumed a that the LLM will output 8 clusters of topics

In [5]:
# Output do LLM será inserido aqui
llm_guess = 8

# Part 2 - NLP 

## Pré-Processamento

In [22]:
# Palavras personalizadas para remoção
custom_words = {
    "alguns", "pouco", "muito", "toda", "todo", "algum", "certo",
    "vários", "nenhum", "tanto", "quanto", "tudo", "nada", "exmo",
    "provimento", "termos", "voto", "exmo", "relator", "juizes", "que",
    "turma", "recursal", "jecs", "unanimidade", "conhecer", "recurso",
    "nos", "termos", "do", "voto", "do", "exmo", "relator", "integram",
    'ilustre', 'dr', ''
}

In [23]:
# Carregar o modelo de linguagem Português
nlp = spacy.load("pt_core_news_sm")
nlp.Defaults.stop_words |= custom_words  # Adicionando stopwords personalizadas


In [24]:
df = pd.read_excel(path)
docs = df.iloc[:, 2].tolist()

### Batches e TF-IDFs

In [25]:
def separar_docs(docs, n):
    """
    Separa os docs em n subcorpora.
    """
    np.random.shuffle(docs)
    tamanho_subcorpus = len(docs) // n
    batches = [
        docs[i * tamanho_subcorpus: (i + 1) * tamanho_subcorpus]
        for i in range(n)
    ]
    return batches

In [26]:
batched_docs = separar_docs(docs, 3)

In [27]:
#Carregar e preprocessar o corpus
def preprocessar(docs, nlp):
    def limpar_texto(texto):
        texto = re.sub(r"<.*?>", "", texto)
        texto = re.sub(r"[^\w\s]", "", texto)
        texto = re.sub(r"\s+", " ", texto)
        texto = re.sub(r"[^\x00-\x7F]+", " ", texto)
        texto = re.sub(r"\d[a|o]", "", texto)
        texto = re.sub(r'(\w+?)(lhe|lhes)\b', r'\1', texto)
        return texto

    def preprocessar_texto(texto):
        texto_limpo = limpar_texto(texto)
        doc = nlp(texto_limpo)
        tokens = [
            token.lower_
            for token in doc
            if not token.is_stop 
            and not token.is_punct
            and not token.like_num
            ]
        return tokens

    corpus_preprocessado = [preprocessar_texto(doc) for doc in docs]
    return corpus_preprocessado

In [28]:
print(preprocessar(
        ['Ilustre representante membro mat dr julgamento publico ministerio fazendaria presente'],
        nlp
    )
)

[['representante', 'membro', 'mat', 'julgamento', 'publico', 'ministerio', 'fazendaria', 'presente']]


In [29]:
processed_docs = [preprocessar(batch, nlp) for batch in batched_docs]

In [30]:
#Vetorizar o corpus
def vetorizar_corpus(corpus_preprocessado):
    corpus_texto = [' '.join(doc) for doc in corpus_preprocessado]
    tfidf_vectorizer = TfidfVectorizer(
        max_df=0.80, 
        min_df=3,
    )
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus_texto)
    return tfidf_matrix, tfidf_vectorizer

In [31]:
tfidf_matrices = [vetorizar_corpus(batch) for batch in processed_docs]

In [32]:
print(tfidf_matrices[0][0])

  (0, 25)	0.08406705943469693
  (0, 0)	0.07600112418162737
  (0, 109)	0.07600112418162737
  (0, 119)	0.07600112418162737
  (0, 22)	0.09927310685642361
  (0, 2)	0.09109769524814217
  (0, 46)	0.07240642634882266
  (0, 29)	0.07417167292176972
  (0, 98)	0.07600112418162737
  (0, 23)	0.08630317714020486
  (0, 114)	0.0778996274294939
  (0, 62)	0.0778996274294939
  (0, 99)	0.0778996274294939
  (0, 118)	0.0778996274294939
  (0, 94)	0.16083435329418125
  (0, 11)	0.15166278308719405
  (0, 51)	0.15166278308719405
  (0, 47)	0.15166278308719405
  (0, 10)	0.15166278308719405
  (0, 111)	0.17205944802846765
  (0, 35)	0.15166278308719405
  (0, 32)	0.3033255661743881
  (0, 3)	0.15166278308719405
  (0, 113)	0.15166278308719405
  (0, 103)	0.15200224836325474
  :	:
  (43, 41)	0.11123163701910763
  (43, 67)	0.12270126501380434
  (43, 61)	0.2835669256278504
  (43, 8)	0.4449265480764305
  (43, 84)	0.11123163701910763
  (43, 50)	0.11394343037713552
  (43, 86)	0.10861178677821057
  (43, 45)	0.11123163701910763


## Aplicação das Técnicas

### NMF

In [33]:
# Aplicar NMF
def aplicar_nmf(tfidf_matrix, n_topics):
    nmf_model = NMF(n_components=n_topics, random_state=42)
    nmf_model.fit(tfidf_matrix)
    return nmf_model

In [34]:
nmf_models = [aplicar_nmf(tfidf_matrix, 3) for tfidf_matrix, _ in tfidf_matrices]

In [35]:
nmf_models

[NMF(n_components=3, random_state=42),
 NMF(n_components=3, random_state=42),
 NMF(n_components=3, random_state=42)]

In [74]:
# Extrair tópicos
def extrair_top_palavras_nmf(nmf_model, tfidf_vectorizer, top_n=10):
    palavras = np.array(tfidf_vectorizer.get_feature_names_out())
    top_palavras = []
    for topic in nmf_model.components_:
        top_palavras.append([palavras[i] for i in topic.argsort()[-top_n:]])
    return top_palavras

In [75]:
top_palavras_nmf = [
    extrair_top_palavras_nmf(
        nmf_models[i],
        tfidf_matrices[i][1]
        ) 
    for i in range(len(nmf_models))
    ]

In [76]:
top_palavras_nmf[0]

[['conforme',
  'condenado',
  'valendo',
  'federal',
  'acordao',
  'recorrente',
  'art',
  'disposto',
  'lei',
  'artigo'],
 ['caput',
  'hipotese',
  'prevista',
  'verificada',
  'dano',
  'onus',
  'sucumbenciais',
  'parcial',
  'artigo',
  'nao'],
 ['cpc',
  'honorarios',
  'custas',
  'negar',
  'legal',
  'julgamento',
  'ministerio',
  'presente',
  'fazendaria',
  'publico']]

In [77]:
# Calcular o Coherence Score
def calcular_coerencia_nmf(top_palavras, corpus_preprocessado, dictionary):
    coherence_model = CoherenceModel(
        topics=top_palavras,
        texts=corpus_preprocessado,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    return coherence_score

In [78]:
nmf_scores = [
    calcular_coerencia_nmf(
        top_palavras_nmf[i],
        processed_docs[i],
        Dictionary(processed_docs[i])
    ) 
    for i in range(len(top_palavras_nmf))
]

In [79]:
nmf_scores

[0.8892691695363256, 0.8632013891759117, 0.9231895296742403]

In [80]:
nmf_topics = [[f"{' '.join(top_palavras_nmf[i][j])}" for j in range(3)] for i in range(len(top_palavras_nmf))]

In [54]:
nmf_whole = [
    {'modelo': nmf_models[i],
     'score': nmf_scores[i],
     'topicos': nmf_topics[i]} 
     for i in range(len(nmf_topics))
]

In [81]:
df_nmf = pd.DataFrame(nmf_whole)
df_nmf

,modelo,score,topicos
0,"NMF(n_components=3, random_state=42)",0.889269,[conforme condenado valendo federal acordao re...
1,"NMF(n_components=3, random_state=42)",0.863201,[honorarios custas manter fundamentos proprios...
2,"NMF(n_components=3, random_state=42)",0.923190,[regimento turmas interno tjrj transcricao dis...


### LDA

In [59]:
# Aplicar LDA
def aplicar_lda(tfidf_matrix, n_topics):
    """
    Aplica o modelo LDA em uma matriz TF-IDF.
    """
    lda_model = LDA(n_components=n_topics, random_state=42)
    lda_topics = lda_model.fit_transform(tfidf_matrix)
    return lda_model, lda_topics

In [61]:
lda_models = [aplicar_lda(tfidf_matrix, 3) for tfidf_matrix, _ in tfidf_matrices]

In [66]:
lda_models

[(LatentDirichletAllocation(n_components=3, random_state=42),
  array([[0.04069785, 0.91835711, 0.04094505],
         [0.33333333, 0.33333333, 0.33333333],
         [0.04385131, 0.91205568, 0.04409302],
         [0.33333333, 0.33333333, 0.33333333],
         [0.04347648, 0.9123354 , 0.04418812],
         [0.53909302, 0.41786813, 0.04303885],
         [0.04048864, 0.91853634, 0.04097502],
         [0.0483901 , 0.90325712, 0.04835278],
         [0.0483901 , 0.90325712, 0.04835278],
         [0.09916226, 0.10050819, 0.80032955],
         [0.06374425, 0.78178512, 0.15447063],
         [0.66482841, 0.16684826, 0.16832333],
         [0.53826938, 0.41949684, 0.04223378],
         [0.50946163, 0.42204951, 0.06848886],
         [0.04348594, 0.91286497, 0.04364909],
         [0.05760581, 0.88549541, 0.05689879],
         [0.04036234, 0.91907387, 0.0405638 ],
         [0.04604119, 0.90264645, 0.05131236],
         [0.08403988, 0.84036282, 0.07559731],
         [0.07982012, 0.09412431, 0.82605557]

In [67]:
def extrair_top_palavras_lda(lda_model, tfidf_vectorizer, top_n=10):
    """
    Extrai as top N palavras de cada tópico do modelo LDA.
    """
    palavras = np.array(tfidf_vectorizer.get_feature_names_out())
    top_palavras = []
    for topic_idx, topic in enumerate(lda_model.components_):
        top_palavras.append([palavras[i] for i in topic.argsort()[-top_n:]])
    return top_palavras

In [83]:
top_palavras_lda = [extrair_top_palavras_lda(lda_models[i][0], tfidf_matrices[i][1]) for i in range(len(lda_models))]

In [84]:
top_palavras_lda[0]

[['mantida',
  'verificada',
  'hipotese',
  'prevista',
  'caput',
  'nao',
  'dano',
  'onus',
  'sucumbenciais',
  'parcial'],
 ['fundamentos',
  'proprios',
  'recorrente',
  'valendo',
  'federal',
  'acordao',
  'disposto',
  'art',
  'lei',
  'artigo'],
 ['custas',
  'cpc',
  'negar',
  'legal',
  'presente',
  'ministerio',
  'fazendaria',
  'julgamento',
  'iv',
  'publico']]

In [86]:
def calcular_coerencia_lda(top_palavras, corpus_preprocessado, dictionary):
    """
    Calcula o Coherence Score para os tópicos gerados pelo LDA.
    """
    coherence_model = CoherenceModel(
        topics=[[word for word in topic if word in dictionary.token2id] 
                for topic in top_palavras],
        texts=corpus_preprocessado,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    return coherence_score


In [87]:
lda_scores = [
    calcular_coerencia_lda(
        top_palavras_lda[i],
        processed_docs[i],
        Dictionary(processed_docs[i])
    ) 
    for i in range(len(top_palavras_lda))
]

In [88]:
lda_scores

[0.8610600604520294, 0.6840876357921323, 0.882489902729031]

In [89]:
lda_topics = [[f"{' '.join(top_palavras_lda[i][j])}" for j in range(3)] for i in range(len(top_palavras_lda))]

In [90]:
lda_whole = [
    {'modelo': lda_models[i],
     'score': lda_scores[i],
     'topicos': lda_topics[i]} 
     for i in range(len(lda_topics))
]

In [92]:
df_lda = pd.DataFrame(lda_whole)
df_lda

,modelo,score,topicos
0,"(LatentDirichletAllocation(n_components=3, ran...",0.861060,[mantida verificada hipotese prevista caput na...
1,"(LatentDirichletAllocation(n_components=3, ran...",0.684088,[fundamentos recorrente principios sendo feder...
2,"(LatentDirichletAllocation(n_components=3, ran...",0.882490,[instancia oficial janeiro rio capital outubro...


### LSA

In [101]:
# Aplicar LSA
def aplicar_lsa(tfidf_matrix, n_topics):
    """
    Aplica o modelo LSA em uma matriz TF-IDF.
    """
    lsa_model = TruncatedSVD(n_components=n_topics, random_state=42)
    lsa_model.fit(tfidf_matrix)
    return lsa_model


In [104]:
lsa_models = [aplicar_lsa(tfidf_matrix, 3) for tfidf_matrix, _ in tfidf_matrices]

In [105]:
lsa_models

[TruncatedSVD(n_components=3, random_state=42),
 TruncatedSVD(n_components=3, random_state=42),
 TruncatedSVD(n_components=3, random_state=42)]

In [102]:
# 4. Extrair tópicos
def extrair_top_palavras_lsa(lsa_model, tfidf_vectorizer, top_n=10):
    """
    Extrai as top N palavras de cada tópico do modelo LSA.
    """
    palavras = np.array(tfidf_vectorizer.get_feature_names_out())
    top_palavras = []
    for topic in lsa_model.components_:
        top_palavras.append([palavras[i] for i in topic.argsort()[-top_n:]])
    return top_palavras

In [106]:
top_palavras_lsa = [extrair_top_palavras_lsa(lsa_models[i], tfidf_matrices[i][1]) for i in range(len(lsa_models))]

In [107]:
top_palavras_lsa[0]

[['fundamentos',
  'resolucao',
  'valendo',
  'recorrente',
  'acordao',
  'federal',
  'art',
  'disposto',
  'lei',
  'artigo'],
 ['caput',
  'verificada',
  'prevista',
  'mantida',
  'parcial',
  'sucumbenciais',
  'onus',
  'dano',
  'nao',
  'artigo'],
 ['iv',
  'negar',
  'legal',
  'parcial',
  'nao',
  'presente',
  'ministerio',
  'fazendaria',
  'julgamento',
  'publico']]

In [103]:
# Calcular o Coherence Score
def calcular_coerencia_lsa(top_palavras, corpus_preprocessado, dictionary):
    """
    Calcula o Coherence Score para os tópicos gerados pelo LSA.
    """
    coherence_model = CoherenceModel(
        topics=[[word for word in topic if word in dictionary.token2id] 
                for topic in top_palavras],
        texts=corpus_preprocessado,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    return coherence_score

In [108]:
lsa_scores = [calcular_coerencia_lsa(top_palavras_lsa[i], processed_docs[i], Dictionary(processed_docs[i])) for i in range(len(top_palavras_lsa))]

In [110]:
lsa_scores

[0.7664024517729856, 0.9037100442133043, 0.9267319723514312]

In [112]:
lsa_topics = [[f"{' '.join(top_palavras_lsa[i][j])}" for j in range(3)] for i in range(len(top_palavras_lsa))] 

In [113]:
lsa_whole = [{'modelo': lsa_models[i], 'score': lsa_scores[i], 'topicos': lsa_topics[i]} for i in range(len(lsa_topics))]

In [115]:
df_lsa = pd.DataFrame(lsa_whole)
df_lsa

,modelo,score,topicos
0,"TruncatedSVD(n_components=3, random_state=42)",0.766402,[fundamentos resolucao valendo recorrente acor...
1,"TruncatedSVD(n_components=3, random_state=42)",0.903710,[resolucao principios sendo recorrente federal...
2,"TruncatedSVD(n_components=3, random_state=42)",0.926732,[valendo federal condenado honorarios custas r...


### Análise da Coerência dos modelos

In [122]:
resultados = {'nmf': df_nmf, 'lda': df_lda, 'lsa': df_lsa}

In [118]:
dfs = []
for modelo_tipo, df in resultados.items():
    df['modelo_tipo'] = modelo_tipo
    dfs.append(df)

# Concatenar todos os dataframes
df_resultados = pd.concat(dfs, ignore_index=True)

df_resultados

## Análise de Coesão dos Grupos

Análise de Coesão dos Grupo -> kmeans.clusters fit(x) {Coesão} -> kmeans.Inertia [evitar negativos] {Distinção} -> kmeans.silluette_score {Relação entre Coesão e Distinção} [coeficiente hand - eval]

Falsos Positivos e Negativos de Cada Tópico -> Justificativa -> Passa por LSA (Importancia Semantica) | pLSA (Probabilistica Importancia Semantica) ->  modelo exato [MUITO IMPORTANTE] | cluster abrangente [POUCO IMPORTANTE]

Calibragem de Modelo -> "Aleatório" -> 